# Import packages

In [14]:
import os
import shutil
from tqdm import tqdm

import math

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from matplotlib import patches as patches
import seaborn as sns
import cv2
from PIL import Image

%matplotlib inline

# Load data

In [15]:
# extract bounding box information into separate col

def extract_nested_list(it):
    if isinstance(it, list):
        for sub_it in it:
            yield from extract_nested_list(sub_it)
    elif isinstance(it, dict):
        for value in it.values():
            yield from extract_nested_list(value)
    else:
        yield it

In [16]:
# Load json files

train_df = pd.read_json("../data/malaria_bbbc_80k/malaria/training.json")
test_df = pd.read_json("../data/malaria_bbbc_80k/malaria/test.json")
print(train_df.shape, test_df.shape)

(1208, 2) (120, 2)


In [17]:
test_df.head()

,image,objects
0,{'checksum': 'eea3bfd6a929bcb06f9786667cd3fbb2...,"[{'bounding_box': {'minimum': {'r': 576, 'c': ..."
1,{'checksum': '2d85fc8ffdb875bb569878243437e4f5...,"[{'bounding_box': {'minimum': {'r': 1048, 'c':..."
2,{'checksum': 'f7c02b6247f3ccfcd4a22c1f06e0c72c...,"[{'bounding_box': {'minimum': {'r': 979, 'c': ..."
3,{'checksum': '50634ef166ae993b4562b96f315ff6db...,"[{'bounding_box': {'minimum': {'r': 402, 'c': ..."
4,{'checksum': '74dd777a74e5a4b24aaee574a99e1459...,"[{'bounding_box': {'minimum': {'r': 314, 'c': ..."


In [18]:
# Create new cols for attributes

train_df["checksum"] = train_df['image'].apply(lambda x: x["checksum"])
train_df["pathname"] = train_df['image'].apply(lambda x: x["pathname"][1:])
train_df["shape"] = train_df['image'].apply(lambda x: x["shape"])
train_df.head()

,image,objects,checksum,pathname,shape
0,{'checksum': '676bb8e86fc2dbf05dd97d51a64ac0af...,"[{'bounding_box': {'minimum': {'r': 1057, 'c':...",676bb8e86fc2dbf05dd97d51a64ac0af,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}"
1,{'checksum': '1225a18efce159eddf7b0e80e0ea642c...,"[{'bounding_box': {'minimum': {'r': 734, 'c': ...",1225a18efce159eddf7b0e80e0ea642c,images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,"{'r': 1200, 'c': 1600, 'channels': 3}"
2,{'checksum': '3eaf840523c30fdf38897ffa01e194eb...,"[{'bounding_box': {'minimum': {'r': 724, 'c': ...",3eaf840523c30fdf38897ffa01e194eb,images/6b14c855-8561-417c-97a4-63fa552842fd.png,"{'r': 1200, 'c': 1600, 'channels': 3}"
3,{'checksum': '8a111dffacfa433029492780b9535091...,"[{'bounding_box': {'minimum': {'r': 563, 'c': ...",8a111dffacfa433029492780b9535091,images/13099edb-35d9-438f-b093-2cf2ebf9d255.png,"{'r': 1200, 'c': 1600, 'channels': 3}"
4,{'checksum': 'ccef403e971460b86444cca669e68ca1...,"[{'bounding_box': {'minimum': {'r': 618, 'c': ...",ccef403e971460b86444cca669e68ca1,images/2559636b-f01a-4414-93da-210c3b12d153.png,"{'r': 1200, 'c': 1600, 'channels': 3}"


In [19]:
# Create new cols for attributes

test_df["checksum"] = test_df['image'].apply(lambda x: x["checksum"])
test_df["pathname"] = test_df['image'].apply(lambda x: x["pathname"][1:])
test_df["shape"] = test_df['image'].apply(lambda x: x["shape"])
test_df.head()

,image,objects,checksum,pathname,shape
0,{'checksum': 'eea3bfd6a929bcb06f9786667cd3fbb2...,"[{'bounding_box': {'minimum': {'r': 576, 'c': ...",eea3bfd6a929bcb06f9786667cd3fbb2,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}"
1,{'checksum': '2d85fc8ffdb875bb569878243437e4f5...,"[{'bounding_box': {'minimum': {'r': 1048, 'c':...",2d85fc8ffdb875bb569878243437e4f5,images/64985a1e-16bb-4016-a01c-c21a5b86e572.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}"
2,{'checksum': 'f7c02b6247f3ccfcd4a22c1f06e0c72c...,"[{'bounding_box': {'minimum': {'r': 979, 'c': ...",f7c02b6247f3ccfcd4a22c1f06e0c72c,images/c70894fa-50e8-4eed-a5a8-159a20153a49.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}"
3,{'checksum': '50634ef166ae993b4562b96f315ff6db...,"[{'bounding_box': {'minimum': {'r': 402, 'c': ...",50634ef166ae993b4562b96f315ff6db,images/abb72ce2-4193-4dd3-a34c-61c2ee22ccff.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}"
4,{'checksum': '74dd777a74e5a4b24aaee574a99e1459...,"[{'bounding_box': {'minimum': {'r': 314, 'c': ...",74dd777a74e5a4b24aaee574a99e1459,images/8db76867-671a-4488-a7b6-991e4dd2e05f.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}"


In [25]:
# create new dataframe with separate bounding boxes as row

df = pd.DataFrame(
    [
     dict(pathname=row['pathname'], 
          shape=row['shape'],
          checksum=row['checksum'],
          **bb_info) 
     for _, row in test_df.iterrows() 
     for bb_info in row['objects']
     ]
     )

df.head()

,pathname,shape,checksum,bounding_box,category
0,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",eea3bfd6a929bcb06f9786667cd3fbb2,"{'minimum': {'r': 576, 'c': 1744}, 'maximum': ...",red blood cell
1,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",eea3bfd6a929bcb06f9786667cd3fbb2,"{'minimum': {'r': 863, 'c': 1249}, 'maximum': ...",red blood cell
2,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",eea3bfd6a929bcb06f9786667cd3fbb2,"{'minimum': {'r': 210, 'c': 1573}, 'maximum': ...",red blood cell
3,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",eea3bfd6a929bcb06f9786667cd3fbb2,"{'minimum': {'r': 1017, 'c': 1124}, 'maximum':...",red blood cell
4,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",eea3bfd6a929bcb06f9786667cd3fbb2,"{'minimum': {'r': 572, 'c': 1655}, 'maximum': ...",red blood cell


In [26]:
# Extract bounding box info to separate cols

df['x_min'] = -1
df['y_min'] = -1
df['x_max'] = -1
df['y_max'] = -1

df[['y_min', 'x_min', 'y_max', 'x_max']] = np.stack(df['bounding_box'].apply(lambda x: np.array(list(extract_nested_list(x)))))

df['x_min'] = df['x_min'].astype(float)
df['y_min'] = df['y_min'].astype(float)
df['x_max'] = df['x_max'].astype(float)
df['y_max'] = df['y_max'].astype(float)

df['w'] = df['x_max'] - df['x_min']
df['h'] = df['y_max'] - df['y_min']

df['x_center'] = df['x_min'] + df['w'] / 2
df['y_center'] = df['y_min'] + df['h'] / 2

df.drop(columns=['bounding_box'], inplace=True)

df.head()

,pathname,shape,checksum,category,x_min,y_min,x_max,y_max,w,h,x_center,y_center
0,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1744.0,576.0,1883.0,708.0,139.0,132.0,1813.5,642.0
1,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1249.0,863.0,1373.0,977.0,124.0,114.0,1311.0,920.0
2,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1573.0,210.0,1711.0,335.0,138.0,125.0,1642.0,272.5
3,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1124.0,1017.0,1253.0,1142.0,129.0,125.0,1188.5,1079.5
4,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1655.0,572.0,1763.0,695.0,108.0,123.0,1709.0,633.5


In [27]:
# Extract image shape info to separate cols

df['img_width'] = -1
df['img_height'] = -1
df['channels'] = -1
df['pixels'] = -1

df[['img_height','img_width', 'channels']] = np.stack(df['shape'].apply(lambda x: np.array(list(extract_nested_list(x)))))
df['pixels'] = df['img_width'] * df['img_height'] * df['channels']

df.drop(columns=['shape'], inplace=True)

df.head()

,pathname,checksum,category,x_min,y_min,x_max,y_max,w,h,x_center,y_center,img_width,img_height,channels,pixels
0,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1744.0,576.0,1883.0,708.0,139.0,132.0,1813.5,642.0,1944,1383,3,8065656
1,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1249.0,863.0,1373.0,977.0,124.0,114.0,1311.0,920.0,1944,1383,3,8065656
2,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1573.0,210.0,1711.0,335.0,138.0,125.0,1642.0,272.5,1944,1383,3,8065656
3,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1124.0,1017.0,1253.0,1142.0,129.0,125.0,1188.5,1079.5,1944,1383,3,8065656
4,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1655.0,572.0,1763.0,695.0,108.0,123.0,1709.0,633.5,1944,1383,3,8065656


In [28]:
# Encoding label

# lb = LabelEncoder()
# lb.fit(df["category"])
df["label"] = lb.transform(df["category"])

df.head()

,pathname,checksum,category,x_min,y_min,x_max,y_max,w,h,x_center,y_center,img_width,img_height,channels,pixels,label
0,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1744.0,576.0,1883.0,708.0,139.0,132.0,1813.5,642.0,1944,1383,3,8065656,3
1,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1249.0,863.0,1373.0,977.0,124.0,114.0,1311.0,920.0,1944,1383,3,8065656,3
2,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1573.0,210.0,1711.0,335.0,138.0,125.0,1642.0,272.5,1944,1383,3,8065656,3
3,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1124.0,1017.0,1253.0,1142.0,129.0,125.0,1188.5,1079.5,1944,1383,3,8065656,3
4,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1655.0,572.0,1763.0,695.0,108.0,123.0,1709.0,633.5,1944,1383,3,8065656,3


In [29]:
lb.classes_

# 7 classes
# difficult: 0, gametocyte: 1, leukocyte: 2, rbc: 3, ring: 4, schizont: 5, trophozoite: 6

array(['difficult', 'gametocyte', 'leukocyte', 'red blood cell', 'ring',
       'schizont', 'trophozoite'], dtype=object)

# Models

## YOLO

### Preprocessing

In [33]:
# convert bb-coordinates for yolo format (center and normalize bounding box coordinates)

df['w_yolo'] = df[['w', 'img_width']].apply(lambda x: x[0] / x[1], axis=1)
df['h_yolo'] = df[['h', 'img_height']].apply(lambda x: x[0] / x[1], axis=1)
df['x_center_yolo'] = df[['x_center', 'img_width']].apply(lambda x: x[0] / x[1], axis=1)
df['y_center_yolo'] = df[['y_center', 'img_height']].apply(lambda x: x[0] / x[1], axis=1)

In [34]:
# get file name

yolov5_df = df.copy()
yolov5_df = yolov5_df[yolov5_df["pathname"].apply(lambda x: x[-4:] == '.jpg')].reset_index(drop=True)
yolov5_df["name"] = yolov5_df["pathname"].apply(lambda x: x.split("/")[-1].split(".")[0])
yolov5_df.head()

,pathname,checksum,category,x_min,y_min,x_max,y_max,w,h,x_center,...,img_width,img_height,channels,pixels,label,w_yolo,h_yolo,x_center_yolo,y_center_yolo,name
0,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1744.0,576.0,1883.0,708.0,139.0,132.0,1813.5,...,1944,1383,3,8065656,3,0.071502,0.095445,0.932870,0.464208,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12
1,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1249.0,863.0,1373.0,977.0,124.0,114.0,1311.0,...,1944,1383,3,8065656,3,0.063786,0.082430,0.674383,0.665221,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12
2,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1573.0,210.0,1711.0,335.0,138.0,125.0,1642.0,...,1944,1383,3,8065656,3,0.070988,0.090383,0.844650,0.197035,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12
3,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1124.0,1017.0,1253.0,1142.0,129.0,125.0,1188.5,...,1944,1383,3,8065656,3,0.066358,0.090383,0.611368,0.780550,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12
4,images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,eea3bfd6a929bcb06f9786667cd3fbb2,red blood cell,1655.0,572.0,1763.0,695.0,108.0,123.0,1709.0,...,1944,1383,3,8065656,3,0.055556,0.088937,0.879115,0.458062,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12


In [35]:
yolov5_df.shape

(5922, 21)

In [37]:
index = list(set(yolov5_df.name))
len(index)

120

In [21]:
source = "images"

for fold in range(5):
    val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
    for name, mini in tqdm(yolov5_df.groupby('name')):
        print(name)
        if name in val_index:
            path2save = 'val/'
        else:
            path2save = 'train/'
        if not os.path.exists('../data/malaria_bbbc_80k/malaria/yolo/fold{}/labels/'.format(fold)+path2save):
            os.makedirs('../data/malaria_bbbc_80k/malaria/yolo/fold{}/labels/'.format(fold)+path2save)
        with open('../data/malaria_bbbc_80k/malaria/yolo/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
            row = mini[['label','x_center_yolo', 'y_center_yolo', 'w_yolo', 'h_yolo']].astype(float).values
            row = row.astype(str)
            for j in range(len(row)):
                text = ' '.join(row[j])
                f.write(text)
                f.write("\n")
        if not os.path.exists('../data/malaria_bbbc_80k/malaria/yolo/fold{}/images/{}'.format(fold,path2save)):
            os.makedirs('../data/malaria_bbbc_80k/malaria/yolo/fold{}/images/{}'.format(fold,path2save))
        shutil.copy("../data/malaria_bbbc_80k/malaria/{}/{}.png".format(source,name),'../data/malaria_bbbc_80k/malaria/yolo/fold{}/images/{}/{}.png'.format(fold,path2save,name))

  0%|▏                                                                                | 2/1208 [00:00<01:00, 19.95it/s]

002f20ad-2ace-499c-9335-c9080bc3e6b5
003a89b0-a095-417a-8dd6-f408339bbc68
003d1cdc-0aec-430b-90e0-f4d82135b1ac
005e60b6-77b8-458c-b57c-bfe0c7e7df78
00a02700-2ea2-4590-9e15-ffc9160fd3de
00c8364b-8c85-4502-bcfe-64736fe76815


  1%|▋                                                                               | 10/1208 [00:00<00:43, 27.66it/s]

00d04a90-80e5-4bce-9511-1b64eabb7a47
0154dd8e-72f8-4d78-a4dd-93e139577bd1
01b38a56-13eb-447b-89ee-fc21cb40dcc1
01b99da8-b66b-464b-a7ee-1cadab8db18b
027ab174-e47a-44f8-b85b-372449eeb7d8
02ea7aa8-c142-4a3d-939f-e77689741341
02fb5e19-6f94-4787-bb78-e550a29a3cbd


  1%|█▏                                                                              | 17/1208 [00:00<00:43, 27.64it/s]

...

f97c7c87-5e27-45ea-9f93-3285d74091de
fa1f9f89-cecb-4972-842c-d14c632f89a9
fa4f8d1a-cb3a-4722-a566-6c7072f30606
fac4b0e2-6b52-4c0e-b762-4fa65d552856
faead1c7-bb2c-4adf-bfce-9ff36cd2cf6a
fb05b297-180a-4171-b633-09d5616d7534
fbb1b44c-d75a-4550-a6c0-0eb62b96bfce


100%|█████████████████████████████████████████████████████████████████████████████▋| 1204/1208 [00:40<00:00, 47.30it/s]

fbc1f360-4382-48bb-a32e-3f5f2f9dbe57
fbcb2ef0-3c97-4d7a-8b96-0147c73f3941
fc15659f-6264-4dd5-ab17-43d89fb297b9
fc8df086-39e0-43f8-b0c7-2d23a349539b
fcc2b245-f47f-447f-8527-275b0005677c
fcdf6632-c03f-49a8-9fb3-42e87c99b9d0
fd1302e2-3cd2-4a1a-9c13-00f810a9c41b
fd1c4c7a-24bc-43f2-b84f-fee182207aee
fde0fd54-49cc-42e1-90c6-6b612cd062d8
fe00fa09-d3c8-485b-9d0f-3cce556c9edd
fe0c5e48-f65e-488c-9c3c-1ea7c7835c0c
fe1ee954-ba71-47b3-954a-d20ab940cd7b
fe436c09-b680-4f4c-8faf-aa10a55a335b
fe691faa-3bc7-475e-b558-aa8fe7b795bc
fe851c88-692d-4199-87e0-d19d9c4eb591
fed6ba05-36a5-45dc-a4e1-9baa7de2c622
ff1b23d0-3cef-459b-b81f-39bfcdb6526f


100%|██████████████████████████████████████████████████████████████████████████████| 1208/1208 [00:40<00:00, 29.46it/s]

ff37dd9b-2e3c-444d-9d29-6db186cfa2fb
ff3b9d4d-373c-48ef-a75a-d23e2c2ade87
ffb7be9a-65df-4f0b-99ae-cfd34e81cd34
ffd59802-46c6-4b58-80fe-e534e39781a7


In [38]:
source = "images"

for name, mini in tqdm(yolov5_df.groupby('name')):
    print(name)

    path2save = 'test/'
    if not os.path.exists('../data/malaria_bbbc_80k/malaria/yolo/labels/'+path2save):
        os.makedirs('../data/malaria_bbbc_80k/malaria/yolo/labels/'+path2save)
    with open('../data/malaria_bbbc_80k/malaria/yolo/labels/'+path2save+name+".txt", 'w+') as f:
        row = mini[['label','x_center_yolo', 'y_center_yolo', 'w_yolo', 'h_yolo']].astype(float).values
        row = row.astype(str)
        for j in range(len(row)):
            text = ' '.join(row[j])
            f.write(text)
            f.write("\n")
    if not os.path.exists('../data/malaria_bbbc_80k/malaria/yolo/images/{}'.format(path2save)):
        os.makedirs('../data/malaria_bbbc_80k/malaria/yolo/images/{}'.format(path2save))
    shutil.copy("../data/malaria_bbbc_80k/malaria/{}/{}.jpg".format(source,name),'../data/malaria_bbbc_80k/malaria/yolo/images/{}/{}.png'.format(path2save,name))

  4%|███▍                                                                              | 5/120 [00:00<00:02, 45.50it/s]

010961af-b38c-49de-aca0-e3732d73d414
0346611b-ea43-45bd-9f16-c6e8086273d2
03be08c8-2cf3-4de6-8ac6-5f7817a6a643
06bbcd42-3ba8-4476-8804-8455f56dd928
0864c569-2ad7-48ed-b12a-e40aa5b93756
087602a2-288b-4a4d-b01a-524f0d799d89
0897f682-5301-49fc-a8d5-9df7762040bb
0a3b53c7-e7ab-4135-80aa-fd2079d727d6
0e0bc36f-6fb2-4763-b0a1-b4c67b2d2d2a
15625643-d964-4b14-9839-c126384ae35b


 12%|██████████▏                                                                      | 15/120 [00:00<00:02, 43.68it/s]

15a5849d-8bc7-4e01-938b-c1f55067787d
1b814efc-7bf9-40ae-8e13-49dc2c04732f
1e647046-333b-4102-a0c6-4c29405c37b3
21116b9d-e6be-483f-8394-e74622b2e182
224c80b8-6c55-48cf-b719-96a47d223dc3
23e581e7-0fb7-42ff-ba55-a86beefc0619
25865b39-56fb-409a-8800-3d368dd264ed
291fe50c-2134-4708-bfa4-09b36ffea123
294bc238-efdb-4f88-b83b-f41fe5750106


 22%|██████████████████▏                                                              | 27/120 [00:00<00:01, 48.96it/s]

...

c736848a-437c-4121-b99f-58f186eb3a1d
c8d1b7ec-be9f-4c88-a39b-2c6bf46b2998
cc453255-b583-425e-bc3e-284ccf120944
cea1cd9b-9b03-407b-ad8d-348155567dab
d1394e1a-dedd-4458-a024-044dae2cd69d
d9255d7f-1282-442d-b152-73d7388526f9
d9ecacb0-14c7-4862-930b-18bb51d5f392
da2197c7-7911-4be6-8bb9-164f47fd270a
dea9006d-c360-4118-b77b-8bc959fa5f8a
deab75c9-00bc-488c-8b20-ff4515d85212


 92%|█████████████████████████████████████████████████████████████████████████▎      | 110/120 [00:02<00:00, 33.61it/s]

df4fd610-fa16-45ca-b3a3-a25a14e5c4e9
e02301ac-68f7-4daf-b2ba-254b917f1cca
e26b88c6-5884-4b5c-94d7-ad780379c8ad
e6a7743e-4040-40fa-b097-ca6874c469b7
e93e617e-ed36-43d9-bd1a-4ffd2fba64e9
ebb53337-1d4f-4ab5-9b6d-83a66f48fe2e
ebc4056e-d766-43cf-b6b4-5dac79d96e84


100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:03<00:00, 39.28it/s]

edea6ca9-3d52-4b4f-bec3-93799b1c26e3
ef55f2ac-5a04-4986-99ab-0d2b5052ab3e
efb85f8e-c780-4a19-b6ce-555a154b0d10
f070a212-e4e2-4743-804c-80789a36f495
f08e873b-9617-42b3-9c7a-55d0e39c2663
f0ee03d6-c57b-43a0-8812-359330bdb93a
f2b1ae99-8169-42ea-bdb4-3351175e4cb6
f8ebe0c8-a90d-4a64-9f6a-f9985e38c340


### Training